Date: February 5, 2021

In [1]:
import pandas as pd
import geopandas as gpd
import momepy as mm
import pyproj
import osmnx
import libpysal
from shapely.geometry import box
from time import time

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-agw853bu because the default path (/home/jovyan/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
osmnx.config(overpass_settings='[out:json][timeout:90][date:"2021-02-05T00:00:00Z"]')

In [3]:
cases = pd.read_csv("data/case_studies.csv")
cases

,case,period,origin
0,Athens,pre-industrial,"(23.729297894645065, 37.977742321097296)"
1,Brugge,pre-industrial,"(3.222135599564842, 51.20663413385126)"
2,Havana,pre-industrial,"(-82.35387869733259, 23.137058811383366)"
3,Kyoto,pre-industrial,"(135.77513789847612, 35.00441991880096)"
4,Nuremberg,pre-industrial,"(11.080741895925831, 49.4558122318182)"
5,Pavia,pre-industrial,"(9.155202505665285, 45.18542787308676)"
6,Recife,pre-industrial,"(-34.87950134898907, -8.066819977715335)"
7,Barcelona,industrial,"(2.1599658311051333, 41.39207228345335)"
8,Brisbane,industrial,"(153.00890689998246, -27.483072513877765)"
9,Buenos Aires,industrial,"(-58.37436109734774, -34.618526518125655)"


In [4]:
%%capture --no-stdout

for i, coords in cases.origin.iteritems():
    s = time()
    """
    donwload and process elements of urban form
    """
    # download buildings from OSM
    point = tuple(map(float, coords[1:-1].split(', ')))[::-1]
    buildings = osmnx.geometries.geometries_from_point(point, dist=800, tags={'building':True})
    buildings = osmnx.projection.project_gdf(buildings)
    buildings = buildings[buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]
    buildings = buildings.reset_index(drop=True).explode().reset_index(drop=True)
    buildings = buildings[["geometry"]]

    # download streets from OSM
    streets_graph = osmnx.graph_from_point(point, 1000, network_type='drive')
    streets_graph = osmnx.get_undirected(streets_graph)
    streets_graph = osmnx.projection.project_graph(streets_graph)
    streets = osmnx.graph_to_gdfs(streets_graph, nodes=False, edges=True,
                                            node_geometry=False, fill_edge_geometry=True)
    streets = streets[["geometry"]]

    # check CRS
    assert buildings.crs.equals(streets.crs)

    # generate tessellation
    buildings['uID'] = range(len(buildings))
    tessellation = mm.Tessellation(buildings, "uID", limit=box(*buildings.total_bounds), verbose=False).tessellation

    """
    measure morphometric characters
    """
    # cell area
    tessellation['cell_area'] = tessellation.area

    # building area
    buildings['blg_area'] = buildings.area

    # coverage area ratio
    tessellation["car"] = mm.AreaRatio(tessellation, buildings, "cell_area", buildings.area, "uID").series

    # segment length
    streets["length"] = streets.length

    # segment linearity
    streets["linearity"] = mm.Linearity(streets, verbose=False).series

    # street profile
    profile = mm.StreetProfile(streets, buildings, distance=3)
    streets["width"] = profile.w
    streets["width_deviation"] = profile.wd
    streets["openness"] = profile.o

    # perimeter wall
    buildings["wall"] = mm.PerimeterWall(buildings, verbose=False).series

    # generate binary contiguity-based W objects of first order and inclusive order 3
    W1 = libpysal.weights.Queen.from_dataframe(tessellation, ids="uID")
    W3 = mm.sw_high(k=3, weights=W1)

    # building adjacency
    buildings["adjacency"] = mm.BuildingAdjacency(buildings, W3, "uID", verbose=False).series

    # interbuilding distance
    buildings['neighbour_distance'] = mm.NeighborDistance(buildings, W1, 'uID', verbose=False).series

    # generate networkx.MultiGraph object
    G = mm.gdf_to_nx(streets)

    # meshedness
    G = mm.meshedness(G, verbose=False)

    # convert networkx.MultiGraph to geopandas.GeoDataFrames
    nodes, edges = mm.nx_to_gdf(G)

    """
    link all characters to tessellation cells
    """
    # merge street network edges based on proximity
    edges["nID"] = range(len(edges))
    buildings["nID"] = mm.get_network_id(buildings, edges, "nID", 500, verbose=False)
    buildings = buildings.merge(edges.drop(columns="geometry"), on="nID", how="left")

    # merge nodes based on edge ID and proximity
    buildings["nodeID"] = mm.get_node_id(buildings, nodes, edges, "nodeID", "nID", verbose=False)
    buildings = buildings.merge(nodes.drop(columns="geometry"), on="nodeID", how="left")

    # merge buildings based on a shared attribute
    tessellation = tessellation.merge(buildings.drop(columns="geometry"), on="uID", how="left")

    """
    save all to GeoPackage
    """
    tessellation.to_file(f"data/{cases.loc[i, 'case']}.gpkg", layer="tessellation", driver="GPKG")
    buildings.to_file(f"data/{cases.loc[i, 'case']}.gpkg", layer="buildings", driver="GPKG")
    edges.to_file(f"data/{cases.loc[i, 'case']}.gpkg", layer="edges", driver="GPKG")
    nodes.to_file(f"data/{cases.loc[i, 'case']}.gpkg", layer="nodes", driver="GPKG")

    print(f"{cases.loc[i, 'case']} done in {time() - s} seconds.")

Athens done in 124.80225110054016 seconds.
Brugge done in 163.39848065376282 seconds.
Havana done in 109.47797226905823 seconds.
Kyoto done in 202.84145283699036 seconds.
Nuremberg done in 82.80549263954163 seconds.
Pavia done in 111.75445604324341 seconds.
Recife done in 94.3385853767395 seconds.
Barcelona done in 162.23757243156433 seconds.
Brisbane done in 68.64858531951904 seconds.
Buenos Aires done in 101.29309368133545 seconds.
Chicago done in 91.5915937423706 seconds.
De Pijp done in 160.9113962650299 seconds.
Paris done in 132.2676899433136 seconds.
Philadelphia done in 259.6676082611084 seconds.
Akademgorodok done in 18.555713891983032 seconds.
Brasilia done in 26.315813541412354 seconds.
Drumul Taberei done in 34.59315204620361 seconds.
Kilamba done in 31.42789053916931 seconds.
Marzhan done in 34.055949449539185 seconds.
Tblisi done in 20.573946237564087 seconds.
Karvina done in 37.80060911178589 seconds.
Ciudad Guayana done in 68.3746988773346 seconds.
Frohnau done in 51.05